# 16-08-28

## Globals

In [2]:
date = "160828"  
directory = "//Andor/share/Data and documents/Data repository/" + date + "/"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit as fit
from matplotlib.ticker import ScalarFormatter
from matplotlib import gridspec
%matplotlib inline

## Fitting Functions

In [3]:
# Fits 
def quadratic(x,a,b,x0):
    return a + b*(x-x0)**2
def gaussian(x, A1, x01, sig1, offset):
    if (offset < 0):
        return 10**10
    return offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2))
def doubleGaussian(x, A1, x01, sig1, A2, x02, sig2, offset):
    if (A1 < 0 or A2 < 0):
        # Penalize negative fits.
        return 10**10
    if (offset < 0):
        return 10**10

    return offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2)) + A2 * np.exp(-(x-x02)**2/(2*sig2**2))
def tripleGaussian(x, A1, x01, sig1, A2, x02, sig2, A3, x03, sig3, offset ):
    if (A1 < 0 or A2 < 0 or A3 < 0):
        # Penalize negative fits.
        return 10**10
    if (offset < 0):
        return 10**10
    return (offset + A1 * np.exp(-(x-x01)**2/(2*sig1**2)) + A2 * np.exp(-(x-x02)**2/(2*sig2**2)) 
            + A3 * np.exp(-(x-x03)**2/(2*sig3**2)))        
def decayingCos(x, A, tau, f, phi, offset):
    if (A < 0):
        return 10**10
    if (phi < 0):
        return 10**10
    if (offset < 0):
        return 10**10
    if (tau > 0):
        return 10**10
    return offset + A * np.exp(-x/tau) * np.cos(2 * np.pi * f * x + phi)

In [4]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

## Import Data Function

In [37]:
def importData(date, name, runNumber):
    """
    This function imports the data created by my first-step python code.
    :param date: the date in YYMMDD format so that the code knows where to look for the data.
    :param name: the name of the compiled data file, minus _run?? (e.g. it's standardized for carrier calibrations
    :param runNumber: The run number.
    :return: returns an OrderedDictionary object from the collections library that contains all the information about
             a run.
    """
    from collections import OrderedDict as dic
    data = dic()
    with open('//andor/share/Data and documents/Data repository/' + str(date) + '/' + str(name) + '_run'
              + str(runNumber) + '.csv') as myfile:
        fileText = myfile.read().replace('\n', '')
        rows = []
        for row in fileText.split(':'):
            rows.append(row.replace('\t', ''))
        del rows[0]
        for itemInc in range(0, len(rows), 2):
            if (rows[itemInc + 1].find(';')) != -1:
                itemData = dic()
                subrows = []
                for subrow in rows[itemInc + 1].split(';'):
                    subrows.append(subrow)
                del subrows[0]
                for subitem in range(0, len(subrows), 2):
                    itemData[subrows[subitem]] = subrows[subitem + 1]
                data[rows[itemInc]] = itemData
            else:
                try:
                    data[rows[itemInc]] = [float(item) for item in (rows[itemInc + 1].split(','))]
                except:
                    data[rows[itemInc]] = rows[itemInc + 1].split(',')
    return data


## #11 Global Microwave

In [38]:
filename = "Carrier_Calibration_Top_Vert"
# DATA ANALYSIS
data11 = importData(date, filename, 11)
print(data11['Key List'])
print(data11['[3, 1]']['Key List'])
print(type(data11['Key']))
print(data11['Key'])
print(min(data11['Key']))

[' Key List', ' Date', ' Run Number', ' Repetitions', ' Pictures Per Experiment', ' Key', ' 3', ' 1', ' 5', ' 1', ' Correlation Averages', ' Correlation Errors']
 Key List, Atom Location, Data Counts, Threshold, Threshold Fidelity, Atoms Data, Survival Averages, Survival Errors, Loading Probabilities
<class 'list'>
[6.841025, 6.840975, 6.841075, 6.840925]
6.840925


In [39]:
filename = "Carrier_Calibration_Top_Vert"
# DATA ANALYSIS
data11 = importData(date, filename, 11)
# fit
print(type(min(data11['Key'])))
xData = np.linspace(min(data11['Key']), max(data11['Key']), 100)
fitValues6, covValues6 = fit(quadratic, data11['Key'], data11['[5, 1]']['Survival Averages'], p0=[1,-5*10**8,6.83096])
fit6Data = quadratic(xData, *fitValues6);
fitValues4, covValues4 = fit(quadratic, data11['Key'], data11['[3, 1]']['Survival Averages'], p0=[1,-5*10**8,6.83096])
fit4Data = quadratic(xData, *fitValues4)
# Carrier Plot
myFig = plt.figure(1, facecolor="white", figsize=(20, 8))
mainPlot = plt.subplot2grid((3,3), (0,0), colspan=2, rowspan=3)
mainPlot.errorbar(keyFromData6, dataPoints6, yerr=error6, ls='', marker='o', label="well 6", color='b', capsize=6,
                  elinewidth=3)
mainPlot.plot(xData, fit6Data, color='b', label='fit 6', linewidth=3)
mainPlot.errorbar(keyFromData4, dataPoints4, yerr=error4, ls='', marker='o', label="well 4", color='r', capsize=6, 
                  elinewidth=3)
mainPlot.plot(xData, fit4Data, color='r', label='fit 4', linewidth=3)
mainPlot.set_ylim({-0.02, 1.01});
mainPlot.set_title("Carrier Calibration", fontsize=30);
mainPlot.set_ylabel("Survival Probability", fontsize=20);
mainPlot.set_xlabel("Carrier Frequency", fontsize=20);
mainPlot.legend(loc="upper center", bbox_to_anchor=(0.5,-0.1), fancybox=True, ncol=4);
mainPlot.grid("on")
# Capture Probabilities Plot
capturePlot = plt.subplot2grid((3,3), (0,2))
capturePlot.plot(key, capture6, ls='', marker='o', color='b')
capturePlot.plot(key, capture4, ls='', marker='o', color='r')
capturePlot.set_ylim({0,1})
capturePlot.set_xlabel("Carrier Frequency")
capturePlot.set_ylabel("Capture %")
capturePlot.set_title("Capture Probabilities")
capturePlot.grid("on")
# Count Series Plot
countDataPlot = plt.subplot2grid((3,3), (1,2))
countDataPlot.plot(count6,'b', ls='', marker='.', markersize=1)
countDataPlot.plot(count4,'r', ls='', marker='.', markersize=1)
countDataPlot.set_xlabel("Picture #")
countDataPlot.set_ylabel("Camera Signal")
countDataPlot.set_title("Camera Signal Over Time")
countDataPlot.grid("on")
# info plot
infoPlot = plt.subplot2grid((3,3), (2,2))
infoPlot.axis("off")
infoPlot.text(0,0, "More info to be (automatically) placed here in the future...")
infoPlot.text(0,0.1,"Accumulations: " + str(content6[0]))
infoPlot.text(0,0.2,"Well 4 Center = " + "{:.7f}".format(fitValues4[2]) + "[" 
              + "{:.0f}".format(10**7*np.sqrt(covValues4[2][2])) + "] GHz", fontweight='bold')
infoPlot.text(0,0.3,"Well 6 Center = " + "{:.7f}".format(fitValues6[2]) + "[" 
              + "{:.0f}".format(10**7*np.sqrt(covValues6[2][2])) + "] GHz", fontweight='bold');
plt.tight_layout()

<class 'float'>


TypeError: ufunc 'subtract' did not contain a loop with signature matching types dtype('<U65') dtype('<U65') dtype('<U65')